In [ ]:
import dask
import dask.array as da
from dask import delayed

In [ ]:
with dask.annotate(priority=100, retries=3):
    A = da.ones((10000, 10000))

In [ ]:
A.visualize()

In [5]:
import distributed
import dask
from dask import delayed

local_cluster = distributed.LocalCluster(n_workers=2)
client = distributed.Client(local_cluster)

workers = list(client.scheduler_info()['workers'].keys())

with dask.annotate(workers=workers[0]):
    delayed_sum = delayed(sum)([1, 2])
    
with dask.annotate(workers=workers[1]):
    doubled = delayed_sum * 2

# use persist so scheduler doesn't clean up
# wrap in a distributed.wait to make sure they're there when we check the scheduler
distributed.wait([doubled.persist(), delayed_sum.persist()])

worker_restrictions = local_cluster.scheduler.worker_restrictions

assert worker_restrictions[delayed_sum.key] == {workers[0]}
assert worker_restrictions[doubled.key] == {workers[1]}

/Users/sarahj/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56689 instead
  warnings.warn(


In [6]:
worker_restrictions

<class 'distributed.scheduler._OptionalStateLegacyMapping'>({})

In [ ]:
import time.sleep

@delayed
def wait_sum():
    sleep(5)
    return sum([1, 2])

In [4]:
distributed.wait?

Signature: distributed.wait(fs, timeout=None, return_when='ALL_COMPLETED')
Docstring:
Wait until all/any futures are finished

Parameters
----------
fs : list of futures
timeout : number, optional
    Time in seconds after which to raise a ``dask.distributed.TimeoutError``
return_when : str, optional
    One of `ALL_COMPLETED` or `FIRST_COMPLETED`

Returns
-------
Named tuple of completed, not completed
File:      ~/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/client.py
Type:      function
